# How to use this notebook

## 1. Up container

```bash
# Build and up container.
$ docker compose build
$ docker compose up
```

## 2. [Optional] If you want to speed up ...

```bash
# If you want to speed up the processsing to evaluate the rank of images ...
# After opening another prompt
$ docker compose exec python bash

# In the docker container,
$ cd torchreid/metrics/rank_cylib
$ pwd
/home/appuser/torchreid/metrics/rank_cylib

# Build the Cython-optimized evaluation method.
$ python setup.py build_ext --inplace
Compiling rank_cy.pyx because it changed.
[1/1] Cythonizing rank_cy.pyx
...(略)...

# Remove the unnecessary directory.
$ rm -rf build
```

## 3. Access notebook!

After that, try to access `localhost:8886` from your Web browser, for example, Google Chrome.


In [1]:
from typing import Dict, List, Tuple


import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torchreid
from torchreid import data, optim, utils, engine, losses, models, metrics

from custom_src.evaluation import CustomEvaluationEngine
from custom_src.ranking_to_csv import CsvCreator


# Data Prepareation

In [2]:
dataset_name: str = "market1501"

# download function は機能していなかったので、手動で deep-person-reid/reid-data/market1501/ の中にデータを設置した
datamanager = torchreid.data.ImageDataManager(
    root="../reid-data",
    sources=dataset_name,
    targets=dataset_name,
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   751 |     3369 |         6
  gallery  |   752 |    15914 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   751 |     3369 |         6
  gal

# Model Preparation

In [3]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [4]:
# 過去に学習した チェックポイント (学習済みパラメータ) を再度利用する
from torchreid.utils import resume_from_checkpoint


# model ファイルのパス
fpath = '../log/resnet50/model/model.pth.tar-60'
start_epoch = resume_from_checkpoint(
    fpath=fpath,
    model=model, optimizer=optimizer, scheduler=scheduler
)

print("start_epoch:", start_epoch)

Loading checkpoint from "../log/resnet50/model/model.pth.tar-60"
Loaded model weights
Loaded optimizer
Loaded scheduler
Last epoch = 60
Last rank1 = 85.6%
start_epoch: 60


In [5]:
# engine = torchreid.engine.ImageSoftmaxEngine(
#     datamanager,
#     model,
#     optimizer=optimizer,
#     scheduler=scheduler,
#     label_smooth=True
# )

# Training

- 実行は一回で良い
- `save_dir`には、学習結果が保存される

In [6]:
# engine.run(
#     save_dir="log/resnet50",
#     max_epoch=60,
#     eval_freq=10,
#     print_freq=10,
#     test_only=False
# )


# Visualize

もともと用意されていた機能

今回行ったカスタマイズ実装があるので、実行する必要はない

In [7]:
# engine.run(
#     save_dir="log/resnet50",
#     max_epoch=60,
#     eval_freq=10,
#     print_freq=10,
#     test_only=True,
#     visrank=True
# )

# Output rank-image and csv

In [5]:
engine = CustomEvaluationEngine(
    datamanager,
    model,
    save_dir="log/resnet50",
    visrank=False
)
rank1_score, distmat = engine.run()
print("shape of distmat:", distmat.shape)
rank1_score

##### Evaluating market1501 (source) #####
Extracting features from query set ...
Done, obtained 3369-by-2048 matrix
Extracting features from gallery set ...
Done, obtained 15914-by-2048 matrix
Speed: 0.0225 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 69.3%
CMC curve
Rank-1  : 85.6%
Rank-5  : 93.7%
Rank-10 : 95.8%
Rank-20 : 97.6%
shape of distmat: (3369, 15914)


0.8557435

In [6]:
CsvCreator().run(distmat, datamanager, dataset_name)

20230908_similarity_ranking.csv is created!
